# Imports

In [1]:
import asyncio

from dotenv import load_dotenv

from saw.workflows.multi_llm.chaining import chain, achain
from saw.workflows.multi_llm.parallelization import parallel, aparallel
from saw.workflows.multi_llm.routing import route, aroute

In [2]:
load_dotenv("../docker/.env")

True

# Variables

In [3]:
provider = "google"
model = "gemini-2.0-flash"
provider2 = "ollama"
model2 = "deepseek-r1:1.5b"
provider3 = "groq"
model3 = "llama3-8b-8192"
provider4 = "openai"
model4 = "gpt-4o-mini"
system_prompt = "You are a helpful assistant."

# Chaining Example
The `chain` function allows us to process a query through multiple prompts sequentially.

In [4]:
query = "What is the best way to master deep learning which is very complex and technical?"

prompts = [
    {
        "prompt": "Provide an overview of the subjects needed for mastering deep learning.",
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [str.lower]  # Example function to lower case the prompt
    },
    {
        "prompt": "Provide a step-by-step guide on how to learn these necessary topics so I can master deep learning.",
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    }
]

In [5]:
# Synchronous chaining
result = chain(query=query, prompts=prompts)

Model: google-gemini-2.0-flash

Step 1: provide an overview of the subjects needed for mastering deep learning.
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=1696 prompt_token_count=37 total_token_count=1733

Result: Mastering deep learning, a complex and technical field, requires a solid foundation across several key subjects. Here's an overview of the essential areas to focus on:

**I. Foundational Mathematics:**

*   **Linear Algebra:** Crucial for understanding how neural networks manipulate data.
    *   *Topics:* Vectors, matrices, tensors, matrix operations (addition, multiplication, transpose, inverse), eigenvalues, eigenvectors, singular value decomposition (SVD), norms, linear transformations.
    *   *Why it's important:* Data representation, optimization algorithms, und

# Parallelization Example
The `parallel` function allows us to process multiple inputs in parallel using a model.

In [6]:
query = "What is the best way to master deep learning which is very complex and technical?"

prompts = [
    {
        "prompt": "What is the best way to learn a complex technical concept in deep learning?",
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    },
    {
        "prompt": "What is the best way to learn a complex technical concept in quantum computing?",
        "model": model2,
        "provider": provider2,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    },
    {
        "prompt": "What are the key differences between supervised and unsupervised learning?",
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    },
    {
        "prompt": "How can I get started with quantum computing as a beginner?",
        "model": model2,
        "provider": provider2,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    }
]

n_workers = 4

In [7]:
# Synchronous parallelization
results = parallel(query=query, prompts=prompts, n_workers=n_workers)

Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=860 prompt_token_count=36 total_token_count=896
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=1560 prompt_token_count=40 total_token_count=1600
Response: dict_keys(['model', 'created_at', 'done', 'done_reason', 'total_duration', 'load_duration', 'prompt_eval_count', 'prompt_eval_duration', 'eval_count', 'eval_duration', 'response', 'context'])
Using model: deepseek-r1:1.5b
Response: dict_keys(['model', 'created_at', 'done', 'done_reason', 'total_duration', 'load_duration', 'prompt_eval_count', 'prompt_eval_duration', 'eval_count', '

In [8]:
results[0]

('what is the best way to learn a complex technical concept in deep learning?',
 'Okay, mastering a complex technical concept in deep learning requires a multi-faceted approach that combines theoretical understanding, practical application, and continuous learning. Here\'s a breakdown of the best strategies:\n\n**1. Foundational Knowledge is Key:**\n\n*   **Build a Strong Math Foundation:** Deep learning relies heavily on linear algebra, calculus, probability, and statistics. Before diving too deep, ensure you have a solid understanding of these areas.\n    *   **Linear Algebra:** Vectors, matrices, matrix operations, eigenvalues, eigenvectors. Understand how data is represented and manipulated.\n    *   **Calculus:** Derivatives (gradients), chain rule, optimization.  Essential for understanding how neural networks learn.\n    *   **Probability & Statistics:** Distributions, hypothesis testing, statistical significance. Important for understanding model performance and avoiding overfi

# Routing Example
The `route` function allows us to route a query to the most appropriate model call based on a selector prompt.

In [9]:
prompt = {
    "prompt": """I'm having trouble understanding some calculus concepts. Can you help explain them or provide some resources?""",
    "model": model,
    "provider": provider,
    "system_prompt": system_prompt,
    "functions": [str.lower]
}

routes = {
    "math_tutor": {
        "prompt": """You are a math tutor. Follow these guidelines:
        1. Always start with "Math Tutor Response:"
        2. Acknowledge the specific math topic
        3. Provide clear explanations and examples
        4. Suggest additional resources or exercises
        5. Encourage the student to ask further questions

        Maintain a supportive and encouraging tone.
        """,
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    },
    "science_tutor": {
        "prompt": """You are a science tutor. Follow these guidelines:
        1. Always start with "Science Tutor Response:"
        2. Acknowledge the specific science topic
        3. Provide clear explanations and examples
        4. Suggest additional resources or experiments
        5. Encourage the student to ask further questions

        Maintain a supportive and encouraging tone.
        """,
        "model": model2,
        "provider": provider2,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    },
    "english_tutor": {
        "prompt": """You are an English tutor. Follow these guidelines:
        1. Always start with "English Tutor Response:"
        2. Acknowledge the specific English topic
        3. Provide clear explanations and examples
        4. Suggest additional resources or exercises
        5. Encourage the student to ask further questions

        Maintain a supportive and encouraging tone.
        """,
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    },
    "history_tutor": {
        "prompt": """You are a history tutor. Follow these guidelines:
        1. Always start with "History Tutor Response:"
        2. Acknowledge the specific history topic
        3. Provide clear explanations and examples
        4. Suggest additional resources or readings
        5. Encourage the student to ask further questions

        Maintain a supportive and encouraging tone.
        """,
        "model": model2,
        "provider": provider2,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    }
}

route_prompt = "Analyze the input and select the most appropriate tutor."

reasoning_prompt = ("Brief explanation of why this question should be routed to a specific tutor. "
                    "Consider key terms, user intent, and urgency level.")

In [10]:
# Synchronous routing
route_result = route(prompt=prompt, reasoning_prompt=reasoning_prompt, route_prompt=route_prompt, routes=routes)


Available routes: ['math_tutor', 'science_tutor', 'english_tutor', 'history_tutor']
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=82 prompt_token_count=134 total_token_count=216

Routing Analysis:
Reasoning: 
The user is asking for help with calculus, which falls under the domain of mathematics. Therefore, the math tutor is the most appropriate selection.

Selection: math_tutor
Model: google-gemini-2.0-flash
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=796 prompt_token_count=100 total_token_count=896
Result: math tutor response:

Hello! I'm here to help you with those tricky 

# Asynchronous Examples
We can also use the asynchronous versions of these functions.

In [11]:
async def async_lowercase(prompt: str) -> str:
    await asyncio.sleep(0.1)
    return prompt.lower()

In [12]:
# Asynchronous Chaining Example with Async Functions
query = "What is the best way to master deep learning which is very complex and technical?"

prompts = [
    {
        "prompt": "Provide an overview of the subjects needed for mastering deep learning.",
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [async_lowercase]  # Example async function to lower case the prompt
    },
    {
        "prompt": "Provide a step-by-step guide on how to learn these necessary topics so I can master deep learning.",
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [async_lowercase]
    }
]

# Define the asynchronous chaining function
async_chain_result = await achain(query=query, prompts=prompts)
print(f"Asynchronous Chaining Result: {async_chain_result}")

Model: google-gemini-2.0-flash

Step 1: provide an overview of the subjects needed for mastering deep learning.
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=1837 prompt_token_count=37 total_token_count=1874

Result: Okay, mastering deep learning is definitely a journey, but breaking it down into subjects makes it much more manageable. Here's an overview of the key areas you'll need to focus on, progressing from foundational knowledge to more advanced topics:

**I. Foundational Mathematics:**

*   **Linear Algebra:** This is *absolutely crucial*.  You need to understand:
    *   **Vectors and Matrices:**  Representing data, operations (addition, multiplication), transposes, inverses.
    *   **Linear Transformations:**  How matrices transform vectors.
    *   **Eigenvalues and Eige

In [13]:
# Asynchronous Parallelization Example with Async Functions
query = "What is the best way to master deep learning which is very complex and technical?"

prompts = [
    {
        "prompt": "What is the best way to learn a complex technical concept in deep learning?",
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [async_lowercase]
    },
    {
        "prompt": "What is the best way to learn a complex technical concept in quantum computing?",
        "model": model2,
        "provider": provider2,
        "system_prompt": system_prompt,
        "functions": [async_lowercase]
    },
    {
        "prompt": "What are the key differences between supervised and unsupervised learning?",
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [async_lowercase]
    },
    {
        "prompt": "How can I get started with quantum computing as a beginner?",
        "model": model2,
        "provider": provider2,
        "system_prompt": system_prompt,
        "functions": [async_lowercase]
    }
]

# Define the asynchronous parallelization function
async_parallel_results = await aparallel(query=query, prompts=prompts)
print(f"Asynchronous Parallelization Results: {async_parallel_results}")

Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=764 prompt_token_count=36 total_token_count=800
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=1615 prompt_token_count=40 total_token_count=1655
Response: dict_keys(['model', 'created_at', 'done', 'done_reason', 'total_duration', 'load_duration', 'prompt_eval_count', 'prompt_eval_duration', 'eval_count', 'eval_duration', 'response', 'context'])
Using model: deepseek-r1:1.5b
Response: dict_keys(['model', 'created_at', 'done', 'done_reason', 'total_duration', 'load_duration', 'prompt_eval_count', 'prompt_eval_duration', 'eval_count', '

In [14]:
# Asynchronous Routing Example with Async Functions
prompt = {
    "prompt": """I'm having trouble understanding some calculus concepts. Can you help explain them or provide some resources?""",
    "model": model,
    "provider": provider,
    "system_prompt": system_prompt,
    "functions": [async_lowercase]
}

routes = {
    "math_tutor": {
        "prompt": """You are a math tutor. Follow these guidelines:
        1. Always start with "Math Tutor Response:"
        2. Acknowledge the specific math topic
        3. Provide clear explanations and examples
        4. Suggest additional resources or exercises
        5. Encourage the student to ask further questions

        Maintain a supportive and encouraging tone.
        """,
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [async_lowercase]
    },
    "science_tutor": {
        "prompt": """You are a science tutor. Follow these guidelines:
        1. Always start with "Science Tutor Response:"
        2. Acknowledge the specific science topic
        3. Provide clear explanations and examples
        4. Suggest additional resources or experiments
        5. Encourage the student to ask further questions

        Maintain a supportive and encouraging tone.
        """,
        "model": model2,
        "provider": provider2,
        "system_prompt": system_prompt,
        "functions": [async_lowercase]
    },
    "english_tutor": {
        "prompt": """You are an English tutor. Follow these guidelines:
        1. Always start with "English Tutor Response:"
        2. Acknowledge the specific English topic
        3. Provide clear explanations and examples
        4. Suggest additional resources or exercises
        5. Encourage the student to ask further questions

        Maintain a supportive and encouraging tone.
        """,
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [async_lowercase]
    },
    "history_tutor": {
        "prompt": """You are a history tutor. Follow these guidelines:
        1. Always start with "History Tutor Response:"
        2. Acknowledge the specific history topic
        3. Provide clear explanations and examples
        4. Suggest additional resources or readings
        5. Encourage the student to ask further questions

        Maintain a supportive and encouraging tone.
        """,
        "model": model2,
        "provider": provider2,
        "system_prompt": system_prompt,
        "functions": [async_lowercase]
    }
}

route_prompt = "Analyze the input and select the most appropriate tutor."

reasoning_prompt = ("Brief explanation of why this question should be routed to a specific tutor. "
                    "Consider key terms, user intent, and urgency level.")

# Define the asynchronous routing function
async_route_result = await aroute(prompt=prompt, reasoning_prompt=reasoning_prompt,
                                  route_prompt=route_prompt, routes=routes)
print(f"Asynchronous Routing Result: {async_route_result}")


Available routes: ['math_tutor', 'science_tutor', 'english_tutor', 'history_tutor']
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=86 prompt_token_count=134 total_token_count=220

Routing Analysis:
Reasoning: 
The user explicitly mentions "calculus concepts," indicating a need for assistance with mathematics.

Selection: math_tutor
Model: google-gemini-2.0-flash
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=305 prompt_token_count=100 total_token_count=405
Result: math tutor response:

Absolutely! I can certainly assist you with calculus concepts. Calculus can be challenging, bu